# Přednáška 9: Bayesovská lineární regrese - řídkost a další předpoklady, aplikace

## Obsah přednášky:

- Předpoklad řídkých regresních koeficientů
- Další možnosti řídkých aprioren
    - Spike and slab prior
    - Horseshoe prior
- Hladkost
    - naivní model
    - Wishart prior
    - Wishart prior s lokalizací kovarianční matice
    - LDL rozklady
- Praktická ukázka

## Bayesovská řídká lineární regrese

Též lze najít pod pojmy "Sparse bayesian learning" nebo "Relevance vector machine". Obdobně jako u metody s regularizací LASSO, naším cílem bude najít v určitém slova smyslu řídké řešení $\beta$: obsahující nuly. 

Vyjdeme opět z modelu dat
$$
f(\mathbf{y}|\beta,\omega)=\mathcal{N}\left(X\beta,\omega^{-1}I_{p}\right),
$$
kde apriorní model $\omega$ volíme jako
$$
f(\omega)= \mathcal{G}\left(c_{0},d_{0}\right).
$$

V předchozím případě jsme měli jednoduchý apriorní model $\beta$ daný jako $f(\beta)= \mathcal{N}\left(\mathbf{0},I_{n}\right)$, u kterého si můžeme všimnout, že kovarianční matice se nám prokopírovala do aposteriorního odhadu jako
$$
\Sigma_{\beta}= \left(\widehat{\omega}X^{T}X+I_{n}\right)^{-1}.
$$
Tato matice je přitom zásadní při tlumení/uvolnění prvků vektoru $\beta$. Zkusíme ji proto modelovat detailněji: jako diagonální matici s neznámými prvky na diagonále:
$$
f(\beta|\mathbf{v})=\mathcal{N}\left(\mathbf{0},\left(\begin{array}{ccc}
v_{1} & 0 & 0\\
0 & \ddots & 0\\
0 & 0 & v_{n}
\end{array}\right)^{-1}\right),
$$
kde $\mathbf{v}$ je neznámý vektor (celou matici budeme značit jako $V$). Je to další neznámý parametr, takže mu přiřadíme apriorno a budeme ho odhadovat. Výpočetně efektivní je volit apriorno konjugované k $f(\beta)$, zvolíme tedy následující gamma rozdělení:
$$
f(v_{j})=\mathcal{G}\left(a_{0},b_{0}\right),\ \forall j.
$$

Model se nám tedy zesložitil, ale např. předpoklady a výpočet pro $\omega$ zůstává nezměněn. Úpravou ale projde výpočet $\beta$ a přibyde výpočet $\mathbf{v}$. 
$$
f(\beta|V)= \mathcal{N}\left(\mathbf{0},V^{-1}\right)=\underbrace{\left(2\pi\right)^{-\frac{n}{2}}}_{coef}\left|V^{-1}\right|^{-\frac{1}{2}}\exp\left(-\frac{1}{2}\beta^TV\beta\right),\\
\sum_j^{n} \ln f(v_j) \propto \sum_{j=1}^{n}(a_{0}-1)\ln v_{j}-\sum_{j=1}^{n}b_{0}v_{j}.
$$

Logaritmus sdružené věrohodnosti vyjádříme jako
$$
\ln f(\mathbf{y},\beta,\omega,V) = \ln|\omega^{-1}I_{p}|^{-\frac{1}{2}} - \frac{1}{2}\omega(\mathbf{y}-X\beta)^T(\mathbf{y}-X\beta) + \ln\left|V^{-1}\right|^{-\frac{1}{2}} - \frac{1}{2}\beta^TV\beta + (c_{0}-1)\ln\omega-d_{0}\omega + \sum_{j=1}^{n}(a_{0}-1)\ln v_{j}-\sum_{j=1}^{n}b_{0}v_{j} + const.
$$

Příslušné marginály:
$$
\tilde{f}(\beta|\mathbf{y}) \propto \exp\left(-\frac{1}{2}\widehat{\omega}(-2)\mathbf{y}^{T}X\beta-\frac{1}{2}\widehat{\omega}\beta^{T}X^{T}X\beta-\frac{1}{2}(\beta^{T}\widehat{V}\beta)\right)\\
\tilde{f}(v_j|\mathbf{y}) \propto \exp\left( (a_{0}-1+\frac{1}{2})\ln v_{j}-\left(b_{0}+\frac{1}{2}\text{diag}\left(\widehat{\beta\beta^{T}}\right)_{j}\right)v_{j} \right),\\
\tilde{f}(\omega|\mathbf{y}) \propto \exp\left( (c_{0}+\frac{p}{2}-1)\ln\omega-(d_{0}+\frac{1}{2}(\mathbf{y}\mathbf{y}^{T}-2\mathbf{y}^{T}X\widehat{\beta}+\widehat{\beta^{T}X^{T}X\beta})\omega \right)
$$
které identifikujeme jako
$$
\tilde{f}(\beta|\mathbf{y})=\mathcal{N}(\mu_{\beta},\Sigma_{\beta}),\\
\tilde{f}(v_j|\mathbf{y})=\mathcal{G}(a_j,b_j),\ \forall j\\
\tilde{f}(\omega|\mathbf{y})=\mathcal{G}(c,d).
$$
a určíme jejich tvarovací parametry $\mu_{\beta},\Sigma_{\beta},a,b,c,d$ jako
\begin{align}
\Sigma_{\beta}= & \left(\widehat{\omega}X^{T}X+\widehat{V}\right)^{-1}\\
\mu_{\beta}= & \Sigma_{\beta}\widehat{\omega}X^{T}\mathbf{y}=\left(\widehat{\omega}X^{T}X+\widehat{V}\right)^{-1}\widehat{\omega}X^{T}\mathbf{y}\\
a_{j}= & a_{0}+\frac{1}{2}\\
b_{j}= & b_{0}+\frac{1}{2}\left(\widehat{\beta\beta^T}\right)_{j,j}\\
c= & c_{0}+\frac{p}{2}\\
d= & d_{0}+\frac{1}{2}\left(\mathbf{y}\mathbf{y}^{T}-2\mathbf{y}^{T}X\widehat{\beta}+\widehat{\beta^{T}X^{T}X\beta}\right)
\end{align}
a ještě doplníme příslušné momenty
$$
\widehat{\beta} = \mu_{\beta},\\
\widehat{\beta\beta^T} = \mu_{\beta}\mu_{\beta}^T + \Sigma_{\beta},\\
\widehat{v}_{j} = \frac{a_j}{b_j},\\
\widehat{\omega} = \frac{c}{d}.
$$

Hotovo, můžeme iterovat.

### Příklad:

Odhad pomocí Bayesovské řídké regrese:

<img src="img_ot/l5_beta_VBsparse.png"> 

<img src="img_ot/l5_beta_VBsparse_v.png"> 

## Rekapitulace výsledků:

|<img src="img_ot/l4_ols_illcond.png"> | <img src="img_ot/l4_tichonov.png"> |
| --- | --- |
| <img src="img_ot/l4_lasso.png"> | <img src="img_ot/l5_beta_VB.png"> | 
| <img src="img_ot/l5_beta_VBsparse.png"> | |


## Bayesovská řídká regrese - další přístupy

Model řídkosti $\beta$ založený na diagonální struktuře koverianční matice s neznámou diagonálou ("Relevance vector machine", "Automatic relevance determination") není samozřejmě jediný, ukážeme si pár dalších. Připoměňme, měli jsme řídké apriorno:
$$
f(\beta|\mathbf{v})=\mathcal{N}\left(\mathbf{0},\left(\begin{array}{ccc}
v_{1} & 0 & 0\\
0 & \ddots & 0\\
0 & 0 & v_{n}
\end{array}\right)^{-1}\right),
$$
kde $\mathbf{v}$ je neznámý odhadovaný vektor (s Gamma apriorním rozdělením). 

### Spike and slab prior  

Máme sadu indikátorových proměnných $\lambda_1,\dots,\lambda_n$ nabývajících hodnot 0 nebo 1 (nebo spojitě něco mezi) a určujících, zda daná složka patří do slab nebo do spike. Příkladem může být např. směs
$$
f(\beta_j|\lambda_j,v_j,\epsilon) = \underbrace{\lambda_j\mathcal{N}(0,v_j^{-1})}_{\text{informativní část}} + \underbrace{(1-\lambda_j)\mathcal{N}(0,\epsilon)}_{\text{šumová část}},
$$
kde $\lambda_j$ má vhodné apriorní rozdělení v intervalu $[0;1]$, např. ořezané exponencielní rozdělení.

### Horseshoe prior

Obdobně jako u předchozího modelu se snažíme modelovat 2 alternetivy modelu, zde pomocí Cauchyho rozdělení, např. jako
$$
f(\beta_j|\lambda_j,\tau) = \mathcal{N}(0,\tau\lambda_j), \\
f(\lambda_j) = \mathcal{C}^{+}(0,1),\\
f(\tau) = \mathcal{C}^{+}(0,\tau_0)
$$
kde $\lambda_j$ je lokální omezení parametru a $\tau$ globální.

## Bayesovská regrese a hladkost

Obdobně jako řídkost, hladkost je jedna z přirozených vlastností, které můžeme po řešení vyžadovat a která dává dobrý praktický význam. Jedna z možností je (obdobně jako u Tichonovského regularizačního členu) modelovat diference. Pokud chceme, aby byl vektor $\beta$ hladký, jeho diference preferujeme relativně malé. Nic nám tedy nebrání aplikovat řídkost na diference:
$$
f(\nabla\beta|V)=\mathcal{N}(\mathbf{0},V^{-1}),
$$
kde $\nabla$ je diskrétní první derivace, $\nabla=\left(\begin{array}{ccc}
1 & -1 & 0\\
0 & \ddots & -1\\
0 & 0 & 1
\end{array}\right)$. 

Z vlastností normálního rozdělení víme: pokud pro regulární matici $C$ platí $f(Cx)=\mathcal{N}(\mu_x,\Sigma_x)$, pak pro $f(x)$ platí: $f(x)=\mathcal{N}(C\mu_x,C\Sigma_xC^T)$.

Z toho nám plyne model pro $\beta$ jako
$$
f(\beta|V)=\mathcal{N}(\nabla \mathbf{0},\nabla^{-1} V^{-1} \nabla^{-1,T}).
$$

Použitím variační Bayesovy metody dostáváme následující tvar aposteriorní střední hodnoty pro $\beta$:
$$
\mu_{\beta} = \left(\widehat{\omega}X^{T}X+\nabla\widehat{V}\nabla^T\right)^{-1}\widehat{\omega}X^{T}\mathbf{y}.
$$
Poznamenejme, že zavedení $\nabla$ ovlivnilo i tvar dalších parametrů, to ovšem teď nebudeme řešit. Řešení:

<img src="img_ot/l6_smoothVB.png">

#### Úkoly:
- o jakou vlastnost jsme tímto modelem přišli? jak to napravit?

### Wishart prior - model plné kovarianční matice

Co kdybychom modelovali plnou kovarianční matici a tím z ní "vytěžili" maximum? Tentokrát tedy budeme předpokládat, že $V$ je plná matice a model $f(\beta)$ doplníme o příslušné apriorno $V$
$$
f(\beta|V) = \mathcal{N}(0,V^{-1}),\\
f(V) = \mathcal{W}(a_0 I_n,b_0) \propto |V|^{b_0-n+1} |a_0 I_n|^{-\frac{b_0}{2}}\exp\left(-\frac{1}{2}\text{trace}\left((a_0 I_n)^{-1}V\right)\right).
$$
Použijeme-li opět variační Bayesovu metodu, dostáváme aposteriorno ve tvaru
$$
\tilde{f}(\beta|\mathbf{y})\propto \exp\left[-\frac{1}{2}\widehat{\omega}(-2)\mathbf{y}^{T}X\beta-\frac{1}{2}\widehat{\omega}\beta^{T}X^{T}X\beta-\frac{1}{2}\left(\beta^{T}\widehat{V}\beta\right)\right],\\
\tilde{f}(V|\mathbf{y})\propto \exp\left[\frac{1}{2}\ln\left|V\right|+\frac{b_{0}-n-1}{2}\ln\left|V\right|-\frac{1}{2}\text{trace}\left(\widehat{\beta\beta^T}V\right)-\frac{1}{2}\text{trace}\left(a_{0}^{-1}I_{n}V\right)\right],\\
$$

kde můžeme identifikovat následující stadardní formy: $\tilde{f}(\beta|\mathbf{y})=\mathcal{N}(\mu_\beta,\Sigma_\beta)$ a $\tilde{f}(V|\mathbf{y})=\mathcal{W}(\Sigma_V,b_V)$. Jejich parametry odvodíme jako
$$
\Sigma_{\beta}= \left(\widehat{\omega} X^{T}X+\widehat{V}\right)^{-1}\\
\mu_{\beta}= \left(\widehat{\omega} X^{T}X+\widehat{V}\right)^{-1}\widehat{\omega} X^{T}\mathbf{y}\\
\Sigma_{V}= \left(\widehat{\beta\beta^T}+\alpha_{0}^{-1}I_{n}\right)^{-1}\\
b_V = b_{0}+1
$$
a doplníme o (nový) moment 
$$
\widehat{V} = b_V\Sigma_V.
$$

Řešení dostáváme následující

<img src="img_ot/l6_VBwish.png">

#### Úkoly:
- Řešení je nestabilní, proč tomu tak je?

### Wishart prior s lokalizací

Tzv. maticová lokalizace je heuristický přístup, který provede defacto oříznutí matice danou maskou, např.

<img src="img_ot/l6_lokalizace.png">

Řešení pak lze zapsat jako úpravu rovnice pro $\Sigma_\beta$ a $\mu_\beta$ (my si ukážeme jen $\mu_\beta$)
$$
\mu_{\beta}= \left(\widehat{\omega} X^{T}X+\widehat{V}\circ Loc\right)^{-1}\widehat{\omega} X^{T}\mathbf{y},
$$
kde $Loc$ je právě maska určující lokalizaci, viz předchozí obrázek. Odhad pak vychází následově

<img src="img_ot/l6_VBwishLoca.png">


### LDL rozklad kovarianční matice (náznak)

Přestože je apriorní model s Wishartem lákavý, jeho stabilita je diskutabilní. Existuje ale jiná možnost jako modelovat určitý tvar kovarianční matice (např. diagonálu a vedlejší sub-diagonály, které v určitém slova smyslu modelují řídkost a hladkost). Takovou možností je využít $LDL^T$ rozkladu, kde model $\beta$ může vypadat následovně
$$
f(\beta|L,D) = \mathcal{N}\left( 0 , \left(LDL^T\right)^{-1} \right),
$$
kde $L$ a $D$ mají následující strukturu
$$
L=\left(\begin{array}{cccc}
1 & 0 & 0 & 0\\
l_{1} & 1 & 0 & 0\\
0 & \ddots & 1 & 0\\
0 & 0 & l_{n-1} & 1
\end{array}\right)\,\,\,\,D=\left(\begin{array}{cccc}
d_1 & 0 & 0 & 0\\
0 & d_2 & 0 & 0\\
0 & \ddots & \ddots & 0\\
0 & 0 & 0 & d_n
\end{array}\right).
$$
To má tu krásnou vlastnost, že výsledná matice je tri-diagonální s tím, její prvky můžeme (zvlášť) modelovat. Řešení je výrazně složitější, ale vede na pěkný výsledek

<img src="img_ot/l6_VBlsapc.png"> <img src="img_ot/l6_VBlsapc_l1.png">

## Rekapitulace výsledků:

|<img src="img_ot/l4_ols_illcond.png"> | <img src="img_ot/l4_tichonov.png"> |
| --- | --- |
| <img src="img_ot/l4_lasso.png"> | <img src="img_ot/l5_beta_VB.png"> | 
| <img src="img_ot/l5_beta_VBsparse.png"> | <img src="img_ot/l6_VBlsapc.png"> |


## Praktická ukázka: odhad zdrojového členu při radiačním úniku

Proč se problémem zabývat?
<img src="img_ot/l6_fuku_obr.jpg" width="400"><img src="img_ot/l6_chernobyl_obr.jpg" width="400">
...ale i drobnější úniky
<img src="img_ot/l6_chernob_fire.jpg" width="400">

### Cíl:
- co a kolik toho uniklo?
- kde to uniklo?
- jaké jsou následky?
<img src="img_ot/l6_depo_chernobyl.jpg" width="400">

### Jak na to? Lineární regresí :-)
- Monitorovací síť ($p$ měření) $\longmapsto$ vektor $\mathbf{y}$.
- Reanalýza počasí + transportní model $\longmapsto$ koeficienty $X_{i,j}$
- Časový průběh úniku $\longmapsto$ vektor $\boldsymbol{\beta}$
- Linearizace
\begin{align*}
y_{1}= & X_{1,1}\beta_{1}+X_{1,2}\beta_{2}+\cdots+X_{1,n}\beta_{n},\\
y_{2}= & X_{2,1}\beta_{1}+X_{2,2}\beta_{2}+\cdots+X_{2,n}\beta_{n},\\
\vdots & \vdots\\
y_{p}= & X_{p,1}\beta_{1}+X_{p,2}\beta_{2}+\cdots+X_{p,n}\beta_{n},
\end{align*}
- Maticový zápis $\mathbf{y}=X\beta$

### Monitorovací síť

<img src="img_ot/l6_suro1.jpg" width="400">

### Transportní model
- reanalýza počasí
<img src="img_ot/l6_gfs-vs-ecmwf.png" width="400">

- model transportu částic a plynů (a jejich chování)

Atmosférický transportní model FLEXPART
<img src="img_ot/l6_flex1.jpg" width="400"><img src="img_ot/l6_flex2.jpg" width="400">

### Případ 1: odhad zdrojového členu Černobylské havárie

- data pokrývají prvních 12 dnů havárie, diskretizace 3h
- simulováno 6 výškových hladin (0 až 3000 metrů)
- simulovány 4 velikosti (0.4, 0.8, 1.2, 5.0 $mu$m)

Monitorovací síť a matice $X$ 
<img src="img_ot/l6_chernob_merici_sit.png" width="400"><img src="img_ot/l6_Cs137fr1hl1.png" width="400">

Výpočet LDL rozkladem:
<img src="img_ot/l6_chernobyl_alg38_all.png" width="800">
<img src="img_ot/l6_chernobyl_alg38_ST.png" width="800">

### Případ 2: únik jódu v roce 2011
- V říjnu 2011 byl zaznamenán zvýšený výskyt jódu I-131 v ovzduší ve střední Evropě.
- 11. listopadu byla informována IAEA, která spustila vyšetřování.
- 17. listopadu byla IAEA informována o zdroji úniku jódu, maďarské továrně na výrobu izotopů (pro lékařské účely).
- Později bylo zjištěno:
    - únik trval od 8. září do 16. listopadu,
    - celkově unikl jód o aktivitě 342 GBq,
    - maximum úniku bylo mezi 12. a 14. říjnem (108 GBq),
    - únik nepřekročil roční povolenou dávku pro danou továrnu.
- Náš cíl: na základě měření tato zjištění potvrdit/vyvrátit.

Monitorovací síť a matice $X$
<img src="img_ot/l6_map_jod2011_topologie.png" width="400"><img src="img_ot/l6_M_example.png" width="400">

### Určení lokace úniku:
- matici $X$ mohu napočítat pro každý bod mapy,
- odhady zdrojového členu $\beta$ pro tyto body
- vyhodnocení "nejlepšího" bodu, např.
    - měření vs. rekonstrukce modelem
    - variační lower bound
$$
f(X_{k}|\mathbf{y})\propto f(X_{k})\times\exp\left(\text{E}[\ln f(\mathbf{y},\theta_{1},\dots,\theta_{q})]-\text{E}[\ln\tilde{f}(\theta_{1})]-\dots-\text{E}[\ln\tilde{f}(\theta_{q})]\right)
$$
<img src="img_ot/l6_map_hys11_alg37.png" width="600">

Odhad zdrojového členu z nejpravděpodobnější lokace:
<img src="img_ot/l6_iodine_source_term.png" width="600">